In [8]:
from tqdm import tqdm
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from helper_functions import accuracy_fn
from safetensors.torch import load_model

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Data Processing

In [6]:
train_df = pd.read_csv("data/EMNIST/raw/emnist-balanced-train.csv")
test_df =  pd.read_csv("data/EMNIST/raw/emnist-balanced-test.csv")

In [11]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, feature_columns, target_column=None, transform=None):
        self.dataframe = dataframe
        self.feature_columns = feature_columns
        self.target_column = target_column
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Get features as a tensor
        features = torch.tensor(self.dataframe.iloc[idx][self.feature_columns].values, dtype=torch.float32, device=device).reshape((1, 28, 28))
        
        # Get target if specified
        if self.target_column is not None:
            target = torch.tensor(self.dataframe.iloc[idx][self.target_column]).type(torch.LongTensor).to(device)
            return features, target
        
        # Return only features if no target column is provided
        return features

In [12]:
train_data = CustomDataset(train_df, train_df.columns[1:], train_df.columns[0])
test_data = CustomDataset(test_df, test_df.columns[1:], test_df.columns[0])

In [13]:
batch_size = 16

# put custom dataset to dataloader
train_dl = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [24]:
# get how many classes are there
classes = pd.unique(train_df["45"].values)
len(classes)

47

In [15]:
# check shapes of dataloader
feature, label = next(iter(train_dl))
feature.shape, label.shape

(torch.Size([16, 1, 28, 28]), torch.Size([16]))

### Load Model

In [27]:
class TinyVGG(nn.Module):

    def __init__(self):
        super().__init__()

        self.block_1 = nn.Sequential(
            nn.Conv2d(1, 10, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(10, 10, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.block_2 = nn.Sequential(
            nn.Conv2d(10, 10, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(10, 10, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(10*7*7, len(classes)+1) # 7 * 7 because maxpool has been done twice which divides the shape of image by 2 twice
        )

    def forward(self, x):
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.classifier(x)
        return x

model = TinyVGG().to(device)
# load_model(model, "tinyvgg.safetensors")

In [28]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.01)

### Train Model

In [29]:
torch.manual_seed(20)

epochs = 5

for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n------")
    
    # TRAINING
    train_loss, train_acc = 0, 0
    model.train()
    for batch, (X, y) in enumerate(train_dl):

        X, y = X.to(device), y.to(device)

        # forward pass
        train_pred = model(X)

        # metrics
        loss = loss_fn(train_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y, y_pred=train_pred.argmax(dim=1))

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # print metrics
    train_loss /= len(train_dl)
    train_acc /= len(train_dl)
    print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc:.2f}%")

    
    # TESTING
    test_loss, test_acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in test_dl:

            X, y = X.to(device), y.to(device)


            # forward pass
            test_pred = model(X)

            # metrics
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))
        
        # print metrics
        test_loss /= len(test_dl)
        test_acc /= len(test_dl)
        print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.2f}%")

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0
------
Train Loss: 0.7406 | Train Accuracy: 77.48%


 20%|██        | 1/5 [01:32<06:10, 92.72s/it]

Test Loss: 0.6495 | Test Accuracy: 79.82%
Epoch: 1
------
Train Loss: 0.4830 | Train Accuracy: 84.12%


 40%|████      | 2/5 [03:01<04:30, 90.30s/it]

...

Test Loss: 0.4663 | Test Accuracy: 84.72%
Epoch: 3
------
Train Loss: 0.4146 | Train Accuracy: 85.91%


 80%|████████  | 4/5 [06:02<01:30, 90.58s/it]

Test Loss: 0.4273 | Test Accuracy: 85.72%
Epoch: 4
------
Train Loss: 0.4024 | Train Accuracy: 86.17%


100%|██████████| 5/5 [07:34<00:00, 90.91s/it]

Test Loss: 0.4396 | Test Accuracy: 85.87%
